In [0]:
path = "/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment"

In [0]:
import pandas as pd
import numpy as np

### **Q_1**

In [0]:
# Read countries silver table, select only the needed columns
countries = pd.read_csv(f"{path}/data_storage/2_silver/silver_finance/silver_countries.csv")
countries = countries[['name', 'alpha-2', 'alpha-3']]
countries = countries.rename(columns={'name' : 'country-name'})

# Read ikea_stores silver table, extract the alpha-2 code from locale column and select only the needed columns
ikea_stores = pd.read_csv(f"{path}/data_storage/2_silver/silver_stores/silver_stores.csv")
ikea_stores['alpha-2'] = ikea_stores['locale'].str[:2]
ikea_stores['alpha-2'] = ikea_stores['alpha-2'].str.upper()
ikea_stores = ikea_stores[['id', 'alpha-2']]
ikea_stores = ikea_stores.drop_duplicates()

# Inner join the two tables
pivot_table = pd.merge(ikea_stores, countries, on='alpha-2', how='inner')
pivot_table = pivot_table.sort_values(by='id')

# Count the number of IKEA stores per country
counts = pivot_table.groupby(['country-name', 'alpha-2', 'alpha-3']).size().reset_index(name='sum_of_IKEA_stores')

display(counts)

# Save the pivot table to a parquet file in gold layer
counts.to_parquet(f"{path}/data_storage/3_gold/Q_1_pivot_table_sum_IKEA_stores.parquet", engine='pyarrow')

### **Q_2**

In [0]:
# Read the big mac silver table, select only the needed columns with data from the last 5 years
big_mac = pd.read_csv(f"{path}/data_storage/2_silver/silver_economist/silver_big-mac-data.csv")
big_mac['date'] = pd.to_datetime(big_mac['date'])
big_mac = big_mac[big_mac['date'] >= pd.to_datetime('today') - pd.DateOffset(years=5)]
big_mac['date'] = big_mac['date'].dt.strftime('%Y-%m-%d')
big_mac = big_mac.rename(columns={'iso_a3' : 'alpha-3'})

# Inner join with table created at Q1 since it contains the countries with IKEA stores
merge_table = pd.merge(big_mac, counts, on='alpha-3', how='inner')

# Create price_USD column
merge_table['price_USD'] = np.where(merge_table['dollar_ex'] != 0, merge_table['local_price'] / merge_table['dollar_ex'], np.nan).round(2)

# Select only needed columns
merge_table = merge_table[['name', 'local_price', 'price_USD','date']]
merge_table = merge_table.drop_duplicates()

# Do average in last 5 years in local and USD price
table_5y_avg_price = merge_table.groupby('name')[['local_price', 'price_USD']].mean().round(2).reset_index()

display(table_5y_avg_price)

# Save the table to a parquet file in gold layer
table_5y_avg_price.to_parquet(f"{path}/data_storage/3_gold/Q_2_table_5y_avg_price.parquet", engine='pyarrow')

### **Q_3**

In [0]:
# Use table from Q2 before doing the average in last 5 years in local and USD price
merge_table['date'] = pd.to_datetime(merge_table['date'])

# Keep only the latest price by ordering the date descending
latest_price = merge_table.sort_values(by=['name', 'date'], ascending=[True, False])
latest_price = latest_price.drop_duplicates(subset='name', keep='first')
latest_price = latest_price[['name', 'local_price', 'price_USD']]

# Save the table to a parquet file in gold layer
display(latest_price)
latest_price.to_parquet(f"{path}/data_storage/3_gold/Q_3_latest_price.parquet", engine='pyarrow')

### **Extra nugget**

In [0]:
# Which countries have an IKEA store but no big mac prices in the data provided
display (counts[~counts['alpha-3'].isin(big_mac['alpha-3'])])
